In [1]:
import pandas as pd
import numpy as np

unknown_injury = 0
df = pd.DataFrame()
for year in range(1975,2019,1):
    per = pd.read_csv('Data/{}/PERSON.csv'.format(year)) #, encoding = "ISO-8859-1")
    try:
        np.sum(per['MAN_REST'] == 1)
    except:
        per['MAN_REST'] = per['REST_USE']
        np.sum(per['MAN_REST'] == 1)
    toddler = per.copy()    
    toddler['Lap Belt'] = 1*((toddler['MAN_REST'] == 2) + (toddler['MAN_REST'] == 3))
    toddler['Shoulder Belt'] = 1*((toddler['MAN_REST'] == 1) + (toddler['MAN_REST'] == 3))
    toddler['Car Seat'] = 1*((toddler['MAN_REST'] == 4) + (toddler['MAN_REST']  >= 10))
    toddler = toddler[toddler['AGE'] >= 2]
    toddler = toddler[toddler['AGE'] <= 6]
    toddler = toddler[toddler['SEAT_POS'] >= 1]  # in car only
    A = toddler.shape[0]
    toddler = toddler[toddler['INJ_SEV'] <= 4]
    unknown_injury = unknown_injury + A - toddler.shape[0]
    toddler['Fatal'] = 1*(toddler['INJ_SEV']==4)
    toddler['Serious Injury+'] = 1*(toddler['INJ_SEV']>=3)
    toddler['Light Injury+'] = 1*(toddler['INJ_SEV']>=2)
    toddler['Minor Injury+'] = 1*(toddler['INJ_SEV']>=1)
    toddler['Front Seat'] = 1*(toddler['SEAT_POS']<=3)
    toddler['Ejected'] = 1*((toddler['EJECTION'] == 1) + (toddler['EJECTION'] == 2))
    toddler['Thrown'] = 1*(toddler['EJECTION'] == 1)

    toddler = toddler[['AGE','SEX','Lap Belt','Shoulder Belt','Car Seat',
                       'Fatal', 'Serious Injury+','Light Injury+', 'Minor Injury+',
                       'Front Seat','Ejected','Thrown']]
    toddler['Year'] = year
    df = pd.concat([df,toddler])    
    print(df.shape)
df.head()

C:\Users\darin\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


(2720, 13)
(5343, 13)
(7990, 13)
(10637, 13)
(13071, 13)
(15234, 13)
(17593, 13)
(19838, 13)
(22178, 13)
(24547, 13)
(27006, 13)
(29642, 13)
(32548, 13)
(35447, 13)
(38475, 13)
(41392, 13)
(44184, 13)
(46958, 13)
(49793, 13)
(52737, 13)
(55827, 13)
(59009, 13)
(61979, 13)
(64926, 13)
(67707, 13)
(70457, 13)
(73167, 13)
(75765, 13)
(78450, 13)
(81114, 13)
(83728, 13)
(86141, 13)
(88309, 13)
(90261, 13)
(92114, 13)
(93917, 13)


C:\Users\darin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(95656, 13)
(97438, 13)
(99213, 13)
(100864, 13)
(102740, 13)
(104699, 13)
(106639, 13)
(108458, 13)


,AGE,SEX,Lap Belt,Shoulder Belt,Car Seat,Fatal,Serious Injury+,Light Injury+,Minor Injury+,Front Seat,Ejected,Thrown,Year
53,4,2,0,0,0,0,1,1,1,1,0,0,1975
61,4,1,0,0,0,1,1,1,1,1,0,0,1975
141,4,1,0,0,0,0,1,1,1,1,0,0,1975
142,2,1,0,0,0,0,1,1,1,1,0,0,1975
143,4,2,0,0,0,0,1,1,1,0,0,0,1975


In [2]:
unknown_injury

705

In [4]:
df.to_csv('Data/toddler.csv', index=False)